<a href="https://colab.research.google.com/github/dpatel200/BIOL3340_Fall2025/blob/main/BIOL3340_finalproject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Bioinformatics Final project (Fall-2025)**

In [1]:
!apt-get -qq update
!apt-get -qq install ncbi-blast+ mafft iqtree hmmer


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Extracting templates from packages: 100%
Selecting previously unselected package fonts-lato.
(Reading database ... 121713 files and directories currently installed.)
Preparing to unpack .../00-fonts-lato_2.0-2.1_all.deb ...
Unpacking fonts-lato (2.0-2.1) ...
Selecting previously unselected package netbase.
Preparing to unpack .../01-netbase_6.3_all.deb ...
Unpacking netbase (6.3) ...
Selecting previously unselected package iqtree.
Preparing to unpack .../02-iqtree_2.0.7+dfsg-1_amd64.deb ...
Unpacking iqtree (2.0.7+dfsg-1) ...
Selecting previously unselected package libclone-perl.
Preparing to unpack .../03-libclone-perl_0.45-1build3_amd64.deb ...
Unpacking libclone-perl (0.45-1build3) ...
Selecting previously unselected package libdata-dump-perl.
Preparing to unpack .../04-libdata-dump-perl_1.25-1_al

In [2]:
!blastp -version
!mafft --version
!hmmscan -h | head -n 3
!iqtree2 -h | head -n 5

blastp: 2.12.0+
 Package: blast 2.12.0, build Mar  8 2022 16:19:08
v7.490 (2021/Oct/30)
# hmmscan :: search sequence(s) against a profile database
# HMMER 3.3.2 (Nov 2020); http://hmmer.org/
# Copyright (C) 2020 Howard Hughes Medical Institute.
IQ-TREE multicore version 2.0.7 for Linux 64-bit built Jan 21 2022
Developed by Bui Quang Minh, Nguyen Lam Tung, Olga Chernomor,
Heiko Schmidt, Dominik Schrempf, Michael Woodhams.

Usage: iqtree [-s ALIGNMENT] [-p PARTITION] [-m MODEL] [-t TREE] ...


# **Project Parameters**

In [3]:
EVAL_THRESHOLD = 1e-10
MIN_ALIGN_LEN = 100
TOP_N = 5

print("Parameters loaded.")


Parameters loaded.


# **Download Demo Proteome (Frankia alni ACN14a)**

In [4]:
!wget -q https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/015/005/GCF_000015005.1_ASM1500v1/GCF_000015005.1_ASM1500v1_protein.faa.gz
!gunzip -f GCF_000015005.1_ASM1500v1_protein.faa.gz

#**Upload Unknown genome**

In [6]:
from google.colab import files
import os

print("Upload your unknown genome (.faa or .fna)...")
uploaded = files.upload()

uploaded_files = list(uploaded.keys())
print("Uploaded:", uploaded_files)

UNKNOWN_FILE = uploaded_files[0]  # use first file


Upload your unknown genome (.faa or .fna)...


Saving F4PNMB_2_rerun_Microbispora_siamensis_5_1_reference.faa to F4PNMB_2_rerun_Microbispora_siamensis_5_1_reference.faa
Uploaded: ['F4PNMB_2_rerun_Microbispora_siamensis_5_1_reference.faa']


#**Detect File Type & Validate FASTA**

In [7]:
def is_fasta(filename):
    with open(filename, "r") as f:
        first = f.readline().strip()
        return first.startswith(">")

if not is_fasta(UNKNOWN_FILE):
    raise ValueError("ERROR: This file does not appear to be FASTA format.")

print("FASTA format confirmed.")

# Protein or nucleotide?
ext = UNKNOWN_FILE.lower()

if ext.endswith(".faa") or ext.endswith(".fa") or ext.endswith(".fasta"):
    TYPE = "protein"
elif ext.endswith(".fna"):
    TYPE = "nucleotide"
else:
    TYPE = "unknown"

print("Detected type:", TYPE)


FASTA format confirmed.
Detected type: protein


#**Predict Proteins if Needed (Prodigal)**

In [8]:
if TYPE == "nucleotide":
    print("Running Prodigal for gene prediction...")
    !prodigal -i $UNKNOWN_FILE -a unknown_predicted.faa -o genes.gbk -q
    PROTEOME = "unknown_predicted.faa"
else:
    PROTEOME = UNKNOWN_FILE

print("Proteome ready:", PROTEOME)


Proteome ready: F4PNMB_2_rerun_Microbispora_siamensis_5_1_reference.faa


#**Choose Proteome Source (Dropdown)**

In [9]:
from ipywidgets import Dropdown

proteome_choice = Dropdown(
    options={
        "Use uploaded unknown genome": PROTEOME,
        "Use demo Frankia proteome": "GCF_000015005.1_ASM1500v1_protein.faa",
    },
    value=PROTEOME,
    description="Proteome:"
)

proteome_choice


Dropdown(description='Proteome:', options={'Use uploaded unknown genome': 'F4PNMB_2_rerun_Microbispora_siamens…

#**Set Selected Proteome**

In [10]:
SELECTED_PROTEOME = proteome_choice.value
print("Selected proteome:", SELECTED_PROTEOME)


Selected proteome: F4PNMB_2_rerun_Microbispora_siamensis_5_1_reference.faa


#**Load Reference Sequences**

In [13]:
%%bash
cat > DivIVA_refs.faa <<EOF
>DivIVA_Acidothermus
MPTSIFGRDMAVDLGTANTLVYVRGRGIVLNEPSVVAINTNTGGILAVGIEAKRMIGRTPGNIVAVRPLK
DGVIADFDTTERMLRYFIQKVHRNRHLARGPRLVVCVPSGCTAVEQRAVKDAGYAAGARRVYIIEEPMAA
AIGAGLPVHEPTGNMVVDIGGGTTEVAVISLGGIVTSQSIRVGGDELDNAIINYVKKEYSLMLGERTAEE
IKMAIGSAFPVPDEPHAEIRGRDLVSGLPKTIVVSAAEIRKAIDEPVNAIIDAVKVTLDKCPPELAGDIM
DRGIVLTGGGALLKGLDERLRHETGMPIHITERPLDSVALGAGKCVEDFDTLQPVLISEPRSR
>DivIVA_Mtb
MPLTPADVHNVAFSKPPIGKRGYNEDEVDAFLDLVENELTRLIEENSDLRQRINELDQELAAGGGAGVTP
QATQAIPAYEPEPGKPAPAAVSAGMNEEQALKAARVLSLAQDTADRLTSTAKAESDKMLADARANAEQIL
GEARHTADATVAEARQRADAMLADAQSRSEAQLRQAQEKADALQADAERKHSEIMGTINQQRTVLEGRLE
QLRTFEREYRTRLKTYLESQLEELGQRGSAAPVDSNADAGGFDQFNRGKN
EOF

cat > MreB_refs.faa <<EOF
>MreB_Acidothermus
MPTSIFGRDMAVDLGTANTLVYVRGRGIVLNEPSVVAINTNTGGILAVGIEAKRMIGRTPGNIVAVRPLK
DGVIADFDTTERMLRYFIQKVHRNRHLARGPRLVVCVPSGCTAVEQRAVKDAGYAAGARRVYIIEEPMAA
AIGAGLPVHEPTGNMVVDIGGGTTEVAVISLGGIVTSQSIRVGGDELDNAIINYVKKEYSLMLGERTAEE
IKMAIGSAFPVPDEPHAEIRGRDLVSGLPKTIVVSAAEIRKAIDEPVNAIIDAVKVTLDKCPPELAGDIM
DRGIVLTGGGALLKGLDERLRHETGMPIHITERPLDSVALGAGKCVEDFDTLQPVLISEPRSR
>MreB_Rubrobacter
MGRVMFGGLFGRDVAIDLGTANTLVYVKGHGIVLSEPSVVAIDTKTDRVVAVGSAAKSMIGRTPGNIVAM
RPLKDGVIADFEVTEKMLSYFIRKVQPKRGFFRSLVGPRVVVCVPSGVTGVELRAVKEATEAAGARQAYT
IEEPLAAAIGAGLPVNEAQGSMVVDIGGGTTEVAVVSLGGIVTKSSIRIAGDDIDDAITNYIQKEYKLAI
GTQTAEQLKIELGSAFRLEEEESAEIRGRDLVTGLPKTVVITSEEVREAISVPVDAIIAAVRDTLDRTPP
ELASDIMDRGMVLVGGGALLRHLDERLRRETGIPVHVADDALMCVAIGSGRCLEEIDSYRSALFAG
EOF

cat > MreC_refs.faa <<EOF
>MreC_Acidothermus
MRDTRRTRLILALLLLTAFTLITLDYQSNGRGVFGDLRRIGLAVFGPVERLAADVVRPVRHAIDTVASFG
SEHKKVQQLQQQVETLRRQLRAQPFEQHRAAELDKLIQVSSIGQYTIVPAQVIAVGRGAGFEWTATIDVG
SRDGVRPEMTVINGDGLVGRVTAVSADTATVVLAIDPQFNVGVRLPNGAIGVVSGHGRAAMSLQLIDPSV
RITPGTGLVTAGSVDQTPFVWGVPVGTVTAVSTPLAGEVQTGTVRPFVDFATLDLVGVVVKPPRTDPRGA
LLASPVPTVTVTVTATPAPVPQPSRPAAQPSPLPSPTPAPSHG
MGRRRTNPTGGLAALFVFTILSLALFTVYVREGDCSEGESCGPLHTVQLGAAEVLGPVQGGVALAASPLG
GLTDRVANVFSGDRNALREELSSSQELAARASQLERENAELRRLLDGERAGYEYAPLARVIAPVGEQLTQ
RVTINVGTADGVGPEQPVIVGENTLVGRTTSRVSRNTAEVMLITDQNFSAGVTIVPPAQFDAASGNVDAS
GTDGEVTYGEGLLRTNIEGYFAVEYVDLSARAEPGDFVITSGRSGGRELLFPPGLLVGTVETASSQDIEQ
YKRIVVSPNVNPSNLEEVRVITGW
EOF

echo "Reference sequences loaded."

Reference sequences loaded.


#**Build BLAST Database**

In [15]:
!makeblastdb -in $SELECTED_PROTEOME -dbtype prot -out unknown_db




Building a new DB, current time: 12/05/2025 13:14:38
New DB name:   /content/unknown_db
New DB title:  F4PNMB_2_rerun_Microbispora_siamensis_5_1_reference.faa
Sequence type: Protein
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 8567 sequences in 0.19338 seconds.




#**Set BLAST Thresholds and Gene List**

In [36]:
# ===== Parameters for BLAST =====
EVAL_THRESHOLD = 1e-5     # Default cutoff for accepting hits (converted to float)
MAX_TARGET_SEQS = 20      # Return top 20 hits (optional adjustable)
GENES = ["DivIVA", "MreB", "MreC"]

print("E-value threshold:", EVAL_THRESHOLD)
print("Genes to search:", GENES)

E-value threshold: 1e-05
Genes to search: ['DivIVA', 'MreB', 'MreC']


#**Run BLAST Searches**

In [37]:
%%bash
# ===== Run BLASTp for DivIVA, MreB, MreC =====

# Use python to print the variable value into bash
EVAL=$(python3 - << 'EOF'
import os
print(os.environ.get("EVAL_THRESHOLD", "1e-5"))
EOF
)

echo "Using e-value threshold: $EVAL"

# Confirm DB exists
if [ ! -e unknown_db.phr ]; then
    echo "ERROR: unknown_db BLAST database not found!"
    echo "Run the makeblastdb cell again."
    exit 1
fi

for gene in DivIVA MreB MreC
do
    echo "→ Searching for $gene homologs..."

    blastp \
        -query ${gene}_refs.faa \
        -db unknown_db \
        -out ${gene}_blast.tsv \
        -evalue "$EVAL" \
        -max_target_seqs 20 \
        -outfmt "6 qseqid sseqid pident length evalue bitscore"

    echo "Finished $gene"
done

echo "BLAST searches completed."

Using e-value threshold: 1e-5
→ Searching for DivIVA homologs...
Finished DivIVA
→ Searching for MreB homologs...
Finished MreB
→ Searching for MreC homologs...
Finished MreC
BLAST searches completed.


#**Filter BLAST Hits**

In [38]:
import pandas as pd

hits = {}

for gene in ["DivIVA", "MreB", "MreC"]:
    df = pd.read_csv(
        f"{gene}_blast.tsv", sep="\t", header=None,
        names=["q", "s", "pid", "len", "evalue", "bit"]
    )
    df = df[(df["evalue"] <= EVAL_THRESHOLD) & (df["len"] >= MIN_ALIGN_LEN)]
    df = df.sort_values("bit", ascending=False).head(TOP_N)

    hits[gene] = df

    print(f"\nTop hits for {gene}:")
    display(df)



Top hits for DivIVA:


,q,s,pid,len,evalue,bit
0,DivIVA_Acidothermus,CGEJBK_06658,79.351,339,0.000000e+00,552.0
1,DivIVA_Acidothermus,CGEJBK_02817,71.217,337,1.170000e-174,486.0
2,DivIVA_Acidothermus,CGEJBK_02049,67.836,342,1.660000e-163,458.0
6,DivIVA_Mtb,CGEJBK_03174,40.892,269,4.490000e-48,159.0
3,DivIVA_Acidothermus,CGEJBK_04044,28.452,239,4.680000e-13,67.8



Top hits for MreB:


,q,s,pid,len,evalue,bit
0,MreB_Acidothermus,CGEJBK_06658,79.351,339,0.000000e+00,552.0
1,MreB_Acidothermus,CGEJBK_02817,71.217,337,1.170000e-174,486.0
2,MreB_Acidothermus,CGEJBK_02049,67.836,342,1.660000e-163,458.0
6,MreB_Rubrobacter,CGEJBK_06658,62.687,335,2.870000e-151,427.0
7,MreB_Rubrobacter,CGEJBK_02817,62.388,335,4.260000e-148,419.0



Top hits for MreC:


,q,s,pid,len,evalue,bit
0,MreC_Acidothermus,CGEJBK_06657,43.103,290,2.570000e-50,178
1,MreC_Acidothermus,CGEJBK_02048,43.125,160,1.510000e-24,102


#**Extract Sequences**

In [45]:
TOP_N = 5
SELECTED_PROTEOME = proteome_paths.get(SELECTED_PROTEOME)

print("TOP_N =", TOP_N)
print("SELECTED_PROTEOME =", SELECTED_PROTEOME)

done

NameError: name 'proteome_paths' is not defined

#**Multiple Sequence Alignment(MAFFT)**

In [32]:
for gene in DivIVA MreB MreC; do
    mafft --auto ${gene}_combined.faa > ${gene}_aligned.faa
done


SyntaxError: invalid syntax (ipython-input-1158803410.py, line 1)

#**Trim Alignments (trimAl)**

In [ ]:
for gene in DivIVA MreB MreC; do
    trimal -in ${gene}_aligned.faa -out ${gene}_trimmed.faa -automated1
done


#**IQ-TREE ML Phylogenies**

In [ ]:
for gene in DivIVA MreB MreC; do
    iqtree -s ${gene}_trimmed.faa -m MFP -nt AUTO
done


#**Visualize Trees**

In [ ]:
from Bio import Phylo
import matplotlib.pyplot as plt

for gene in ["DivIVA", "MreB", "MreC"]:
    treefile = f"{gene}_trimmed.faa.treefile"
    print(f"Tree: {gene}")
    plt.figure(figsize=(10,8))
    Phylo.draw(Phylo.read(treefile, "newick"))


#**Sequence Logos**

In [ ]:
for gene in DivIVA MreB MreC; do
    weblogo -f ${gene}_aligned.faa -o ${gene}_logo.png -F png --errorbars NO
done


#**Display Logos**

In [ ]:
from IPython.display import Image, display

for gene in ["DivIVA", "MreB", "MreC"]:
    print(f"{gene} logo:")
    display(Image(f"{gene}_logo.png"))